## Datasets
-  ImageNet
    - data수 : 1.2M / 50K
    - class : 1000
    - class당 data : 1200 개 (train set)
- stanford_dogs
    - data수 : 1.2K / 8580
    - class : 120
    - class당 data : 100개 (train set)
- CIFAR-100
    - data수 : 50K / 10K
    - class : 100 (superclass 존재함)
    - class당 data : 500개 (train set)
- CIFAR-10
    - data수 : 50K / 10K
    - class : 10
    - class당 data : 5000개 (train set)

- __CIFAR-100이 ImageNet과 stanford_dogs의 class당 data수가 중간쯤 되니 이거로 쌩학습 ㄱㄱ__

In [ ]:
/home/aiffel/KangTae123/aiffel/aug_test_cifar.ipynb

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import numpy as np

import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

import pickle


In [ ]:
BATCH_SIZE = 16

## Cutmix

In [ ]:
def get_clip_box(image_a, image_b, proba = 1.0):
    # image.shape = (height, width, channel)
    image_size_x = image_a.shape[1]
    image_size_y = image_a.shape[0]
    
    # get center of box
    x = tf.cast( tf.random.uniform([],0, image_size_x),tf.int32)
    y = tf.cast( tf.random.uniform([],0, image_size_y),tf.int32)
    
    P = tf.cast( tf.random.uniform([],0,1)<=proba, tf.int32)
    
    # get width, height of box
    width = tf.cast(image_size_x * tf.math.sqrt(1-tf.random.uniform([],0,1)),tf.int32) * P
    height = tf.cast(image_size_y * tf.math.sqrt(1-tf.random.uniform([],0,1)),tf.int32) * P
    
    # clip box in image and get minmax bbox
    xa = tf.math.maximum(0, x-width//2)
    ya = tf.math.maximum(0, y-height//2)
    xb = tf.math.minimum(image_size_x, x+width//2)
    yb = tf.math.minimum(image_size_y, y+width//2)
    
    return xa, ya, xb, yb

# mix two images
def mix_2_images(image_a, image_b, xa, ya, xb, yb):
    image_size_x = image_a.shape[1]
    image_size_y = image_a.shape[0] 
    one = image_a[ya:yb,0:xa,:]
    two = image_b[ya:yb,xa:xb,:]
    three = image_a[ya:yb,xb:image_size_x,:]
    middle = tf.concat([one,two,three],axis=1)
    
    top = image_a[0:ya,:,:]
    bottom = image_a[yb:image_size_y,:,:]
    mixed_img = tf.concat([top, middle, bottom],axis=0)
    return mixed_img

def mix_2_label(image_a, image_b, label_a, label_b, xa, ya, xb, yb, num_classes=100):
    image_size_x = image_a.shape[1]
    image_size_y = image_a.shape[0] 
    mixed_area = (xb-xa)*(yb-ya)
    total_area = image_size_x*image_size_y
    a = tf.cast(mixed_area/total_area, tf.float32)

    if len(label_a.shape)==0:
        label_a = tf.one_hot(label_a, num_classes)
    if len(label_b.shape)==0:
        label_b = tf.one_hot(label_b, num_classes)
    mixed_label = a*label_a + (1-a)*label_b
    return mixed_label

def cutmix(image, label, prob = 0.666, batch_size=BATCH_SIZE, img_size=32, num_classes=100, proba = 1.0):
    mixed_imgs = []
    mixed_labels = []

    for i in range(batch_size):
        image_a = image[i]
        label_a = label[i]
        j = tf.cast(tf.random.uniform([],0, batch_size),tf.int32)
        image_b = image[j]
        label_b = label[j]
        
        xa, ya, xb, yb = get_clip_box(image_a, image_b, proba)
        mixed_imgs.append(mix_2_images(image_a, image_b, xa, ya, xb, yb))
        mixed_labels.append(mix_2_label(image_a, image_b, label_a, label_b, xa, ya, xb, yb))

    mixed_imgs = tf.reshape(tf.stack(mixed_imgs),(batch_size, img_size, img_size, 3))
    mixed_labels = tf.reshape(tf.stack(mixed_labels),(batch_size, num_classes))
    return mixed_imgs, mixed_labels

## MixUp

In [ ]:
# function for mixup
def mixup_2_images(image_a, image_b, label_a, label_b):
    a = tf.random.uniform([],0,1)
    
    if len(label_a.shape)==0:
        label_a = tf.one_hot(label_a, num_classes)
    if len(label_b.shape)==0:
        label_b = tf.one_hot(label_b, num_classes)
    mixed_image= (1-a)*image_a + a*image_b
    mixed_label = (1-a)*label_a + a*label_b
    
    return mixed_image, mixed_label

def mixup(image, label, prob = 1.0, batch_size=BATCH_SIZE, img_size=32, num_classes=100):
    mixed_imgs = []
    mixed_labels = []

    for i in range(batch_size):
        image_a = image[i]
        label_a = label[i]
        j = tf.cast(tf.random.uniform([],0, batch_size),tf.int32)
        image_b = image[j]
        label_b = label[j]
        mixed_img, mixed_label = mixup_2_images(image_a, image_b, label_a, label_b)
        mixed_imgs.append(mixed_img)
        mixed_labels.append(mixed_label)

    mixed_imgs = tf.reshape(tf.stack(mixed_imgs),(batch_size, img_size, img_size, 3))
    mixed_labels = tf.reshape(tf.stack(mixed_labels),(batch_size, num_classes))
    return mixed_imgs, mixed_labels


## dataset 전처리

    
- official code의 norm 적용 (https://github.com/tensorflow/tensorflow/blob/2b96f3662bd776e277f86997659e61046b56c315/tensorflow/python/keras/applications/imagenet_utils.py#L158)

In [ ]:
BATCH_SIZE = 16

# def normalize_and_resize_img_origin(image, label):
#     """Normalizes images: `uint8` -> `float32`."""
#     image = tf.image.resize(image, [224, 224])
#     return tf.cast(image, tf.float32) / 255., label

# official code normalize (imagenet & cifar-100)
def normalize_and_resize_img(image, label, mean=[125.3, 123.0, 113.9], std=[63.0, 62.1, 66.7]):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [32, 32])
    R = tf.divide(tf.subtract(image[..., 0], mean[0]), std[0])
    G = tf.divide(tf.subtract(image[..., 1], mean[1]), std[1])
    B = tf.divide(tf.subtract(image[..., 2], mean[2]), std[2])
    
    return tf.cast(tf.stack([R,G,B], axis=-1), tf.float32), label

In [ ]:
def base_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.central_crop(image, np.random.uniform(0.50, 1.00))
    image = tf.image.resize(image, [32, 32])

    return image, label

- augment 는 이전과같이 baseline으로 적용

In [ ]:
def onehot(image, label):
    num_classes = 100
    onehot_label = tf.one_hot(label, num_classes)
    return image, onehot_label

def apply_normalize_on_dataset(ds, is_test=False, batch_size=BATCH_SIZE, with_aug=False, with_cutmix=False, with_mixup=False):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    
    ds = ds.batch(batch_size)
    
    if not is_test and  with_aug:
        ds = ds.map(
            base_augment,
            num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
    
    if not is_test and with_cutmix:
        ds = ds.map(
            cutmix,
            num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
        
    elif not is_test and with_mixup:
        ds = ds.map(
            mixup,
            num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
    
    elif not with_cutmix and not with_mixup and is_test:
        ds = ds.map(
            onehot,
            num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
        
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
import urllib3
urllib3.disable_warnings()
(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)

num_classes = ds_info.features["label"].num_classes
num_classes

In [ ]:
ds_train_cutmix = apply_normalize_on_dataset(ds_train, with_aug=True, with_cutmix=True, with_mixup=False)
ds_train_mixup = apply_normalize_on_dataset(ds_train, with_aug=True, with_cutmix=False, with_mixup=True)

ds_test = apply_normalize_on_dataset(ds_test, is_test=True)


(ds_train_base, ds_test_base), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train_base = apply_normalize_on_dataset(ds_train_base, with_aug=True, with_cutmix=False, with_mixup=False)
ds_test_base = ds_test_base.map(normalize_and_resize_img, 
        num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)



EPOCH = 30

## basemodel

In [ ]:
# augmentation 적용 할 모델 (bsseline)

resnet50_baseline = keras.models.Sequential([
    keras.applications.resnet.ResNet50(
        include_top=False,
        input_shape=(32, 32,3),
        pooling='avg',
    ),
    keras.layers.Dense(num_classes, activation = 'softmax')
])

- base는 SGD 모멘텀, decay있는게 3에폭동안 수렴 좋음

In [ ]:
tf.random.set_seed(2020)
resnet50_baseline.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['acc']
)

resnet50_base_hist = resnet50_baseline.fit(
    ds_train_base, # augmentation 적용하지 않은 데이터셋 사용
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test_base,
    verbose=1,
    use_multiprocessing=True,
)

resnet50_baseline.save_weights('./cifar100_aug/resnet50_baseline')
with open('./cifar100_aug/resnet50_base_hist', 'wb') as file_pi:
    pickle.dump(resnet50_base_hist.history, file_pi)

In [ ]:
resnet50_base_hist = pickle.load(open('./cifar100_aug/resnet50_base_hist', "rb"))

In [ ]:
plt.plot(resnet50_base_hist['loss'], 'r')
plt.plot(resnet50_base_hist['val_loss'], 'b')
plt.title('baseline loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(resnet50_base_hist['sparse_categorical_accuracy'], 'r')
plt.plot(resnet50_base_hist['val_sparse_categorical_accuracy'], 'b')
plt.title('baseline categoical_accuracy')
plt.ylabel('ACC')
plt.xlabel('Epoch')
plt.legend(['train_accuracy', 'val_accuracy'], loc='upper left')
plt.show()

## Cutmix model

In [ ]:
resnet50_cutmix = keras.models.Sequential([
    keras.applications.resnet.ResNet50(
        include_top=False,
        input_shape=(32, 32,3),
        pooling='avg',
    ),
    keras.layers.Dense(num_classes, activation = 'softmax')
])

In [ ]:
resnet50_cutmix.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['acc']
)

resnet50_cutmix_hist = resnet50_cutmix.fit(
    ds_train_cutmix, # augmentation 적용하지 않은 데이터셋 사용
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

resnet50_cutmix.save_weights('./cifar100_aug/resnet50_cutmix')
with open('./cifar100_aug/resnet50_cutmix_hist', 'wb') as file_pi:
    pickle.dump(resnet50_cutmix_hist.history, file_pi)


In [ ]:
resnet50_cutmix_hist = pickle.load(open('./cifar100_aug/resnet50_cutmix_hist', "rb"))

In [ ]:
plt.plot(resnet50_cutmix_hist['loss'], 'r')
plt.plot(resnet50_cutmix_hist['val_loss'], 'b')
plt.title('cutmix model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(resnet50_cutmix_hist['categorical_accuracy'], 'r')
plt.plot(resnet50_cutmix_hist['val_categorical_accuracy'], 'b')
plt.title('cutmix model categoical_accuracy')
plt.ylabel('ACC')
plt.xlabel('Epoch')
plt.legend(['train_accuracy', 'val_accuracy'], loc='upper left')
plt.show()

In [ ]:
plt.plot(resnet50_cutmix_hist['val_loss'], 'r')
plt.plot(resnet50_base_hist['val_loss'], 'b')
plt.title('cutmix vs base loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['cutmix loss', 'base loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(resnet50_cutmix_hist['val_categorical_accuracy'], 'r')
plt.plot(resnet50_base_hist['val_sparse_categorical_accuracy'], 'b')
plt.title('cutmix vs base acc')
plt.ylabel('ACC')
plt.xlabel('Epoch')
plt.legend(['cutmix acc', 'base acc'], loc='upper left')
plt.show()

## MixUp model

In [ ]:
resnet50_mixup = keras.models.Sequential([
    keras.applications.resnet.ResNet50(
        include_top=False,
        input_shape=(32, 32,3),
        pooling='avg',
    ),
    keras.layers.Dense(num_classes, activation = 'softmax')
])

In [ ]:
resnet50_mixup.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['acc']
)

resnet50_mixup_hist = resnet50_mixup.fit(
    ds_train_mixup, # augmentation 적용하지 않은 데이터셋 사용
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

resnet50_mixup.save_weights('./cifar100_aug/resnet50_mixup')
with open('./cifar100_aug/resnet50_mixup_hist', 'wb') as file_pi:
    pickle.dump(resnet50_mixup_hist.history, file_pi)

resnet50_mixup_hist = pickle.load(open('./cifar100_aug/resnet50_mixup_hist', "rb"))

In [ ]:
resnet50_mixup_hist = pickle.load(open('./cifar100_aug/resnet50_mixup_hist', "rb"))

In [ ]:
plt.plot(resnet50_mixup_hist['loss'], 'r')
plt.plot(resnet50_mixup_hist['val_loss'], 'b')
plt.title('mixup model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(resnet50_mixup_hist['categorical_accuracy'], 'r')
plt.plot(resnet50_mixup_hist['val_categorical_accuracy'], 'b')
plt.title('mixup model categoical_accuracy')
plt.ylabel('ACC')
plt.xlabel('Epoch')
plt.legend(['train_accuracy', 'val_accuracy'], loc='upper left')
plt.show()